# day 25

https://adventofcode.com/2021/day/25

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day25.txt')

LOGGER = logging.getLogger('day25')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
def parse_map(m):
    easties = set()
    southies = set()
    for (i, line) in enumerate(m.split('\n')):
        for (j, char) in enumerate(line.strip()):
            if char == '>':
                easties.add((i, j))
            elif char == 'v':
                southies.add((i, j))
            elif char != '.':
                raise ValueError("Unknown character")
    return easties, southies, i + 1, j + 1

#### function def

In [ ]:
def step(i, j, direction, N):
    if direction == '>':
        return i, (j + 1) % N
    elif direction == 'v':
        return (i + 1) % N, j
    else:
        raise ValueError("Unknown direction")

In [ ]:
class Herds:
    def __init__(self, easties, southies, n_i, n_j):
        self.cucumbers = {'>': easties, 'v': southies}
        self.N = {'>': n_j, 'v': n_i}
    
    @property
    def all_cucumber_locations(self):
        return set(self.cucumbers['>']).union(self.cucumbers['v'])
    
    def step(self):
        num_moves = 0
        for direction in '>v':
            moves = self.get_valid_moves(direction)
            num_moves += len(moves)
            self.move(moves, direction)
        return num_moves
    
    def get_valid_moves(self, direction):
        moves = []
        for cuc in self.cucumbers[direction]:
            cuc_next = step(*cuc, direction, self.N[direction])
            if cuc_next not in self.all_cucumber_locations:
                moves.append((cuc, cuc_next))
        return moves
    
    def move(self, moves, direction):
        for (cuc, cuc_next) in moves:
            self.cucumbers[direction].remove(cuc)
            self.cucumbers[direction].add(cuc_next)
    
    def show(self):
        for i in range(self.N['v']):
            s = ''
            for j in range(self.N['>']):
                s += ('>' if (i, j) in self.cucumbers['>']
                      else 'v' if (i, j) in self.cucumbers['v']
                      else '.')
            print(s)

In [ ]:
m = '...>>>>>...'
easties, southies, n_i, n_j = parse_map(m)
h = Herds(easties, southies, n_i, n_j)
print('inital setup')
h.show()
print()

num_moves = h.step()
print(f'after one step (num_moves = {num_moves})')
h.show()
print()

num_moves = h.step()
print(f'after two steps (num_moves = {num_moves})')
h.show()

In [ ]:
m = """..........
.>v....v..
.......>..
.........."""
h = Herds(*parse_map(m))

print('inital setup')
h.show()
print()

num_moves = h.step()
print(f'after one step (num_moves = {num_moves})')
h.show()
print()

num_moves = h.step()
print(f'after two steps (num_moves = {num_moves})')
h.show()
print()

num_moves = h.step()
print(f'after three steps (num_moves = {num_moves})')
h.show()

In [ ]:
h = Herds(*parse_map(test_data))

print("Initial state:")
h.show()
print()

print_steps = [1, 2, 3, 4, 5,
               10, 20, 30, 40, 50,
               55, 56, 57, 58]

for i in range(1, 59):
    num_moves = h.step()
    if i in print_steps:
        print(f"After {i} steps")
        h.show()
        print()

In [ ]:
def q_1(data):
    h = Herds(*parse_map(data))
    step_num = 0
    while True:
        num_moves = h.step()
        step_num += 1
        LOGGER.debug(f"step_num = {step_num}, num_moves = {num_moves}")
        if num_moves == 0:
            return step_num
        if step_num % 10 == 0:
            LOGGER.info(f'step_num = {step_num}')

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 58
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    return False

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == True
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

In [ ]:
#!/usr/bin/python3
import sys
import heapq
import itertools
import re
import ast
from collections import defaultdict, Counter, deque
from copy import deepcopy

data = load_data()

G = []
for line in data.split('\n'):
    assert line.strip() == line
    G.append(line)
R = len(G)
C = len(G[0])

t = 0
while True:
    t += 1
    moved = False
    G2 = [[G[r][c] for c in range(C)] for r in range(R)]
    for r in range(R):
        for c in range(C):
            if G[r][c] == '>':
                if G[r][(c+1)%C] == '.':
                    moved = True
                    G2[r][(c+1)%C] = '>'
                    G2[r][c] = '.'
    G3 = [[G2[r][c] for c in range(C)] for r in range(R)]
    for r in range(R):
        for c in range(C):
            if G2[r][c] == 'v' and G2[(r+1)%R][c] == '.':
                moved = True
                G3[(r+1)%R][c] = 'v'
                G3[r][c] = '.'
    if not moved:
        print(t)
        break
    G = G3